# Python 读写 CSV 文件

本笔记系统整理 Python 中处理 **CSV (逗号分隔文本)** 的常见与进阶方式，涵盖：内置 `csv` 模块、`pandas`、分隔符/编码/换行/引用、自动检测、批量/流式处理、压缩、常见错误与性能优化、安全风险防护 (CSV 注入)、以及一些实战模式。

```shell
pip install pandas
```

> 核心结论速览：小而简单的 CSV 用内置 `csv`；分析/清洗/批处理用 `pandas`；巨型文件优先流式/分块；注意编码与公式注入。

## 1. CSV 基础与文件结构
一个典型 CSV：首行标题，后续行为数据，字段由分隔符 (默认逗号) 分隔，行以换行结束。
示例：
```
姓名,年龄,城市
张三,25,北京
李四,30,上海
王五,28,广州
```
常见变体：
- 分隔符：逗号(,), 制表符(\t), 分号(;), 竖线(|) 等
- 引用：值含分隔符或换行时需用引号包裹 ("上海,中国")
- 编码：UTF-8 (推荐)、GBK (国内旧系统)、UTF-8-BOM (Excel 兼容)
- 行尾：Windows (\r\n)、Unix (\n)


## 2. 使用内置 csv 模块读取基础 CSV
`csv.reader` 逐行返回列表；用 `newline=''` 避免空行问题 (Windows)，显式声明编码。

In [17]:
import csv

with open('sample.csv', 'r', encoding='utf-8', newline='') as f:
    reader = csv.reader(f)
    header = next(reader)  # 读取表头
    print('表头:', header)
    for row in reader:
        print(row)  # row 是列表

表头: ['姓名', '年龄', '城市']
['张三', '25', '北京']
['李四', '30', '上海']
['王五', '28', '广州']


### 2.1 使用 DictReader 读取为字典
方便按列名访问。

In [18]:
import csv

with open('sample.csv', 'r', encoding='utf-8', newline='') as f:
    reader = csv.DictReader(f)
    print("type of reader:",type(reader))
    print(reader.fieldnames)
    for row in reader:
        print(row['姓名'],row['年龄'], row['城市'])

type of reader: <class 'csv.DictReader'>
['姓名', '年龄', '城市']
张三 25 北京
李四 30 上海
王五 28 广州


## 3. 写入 CSV：writer 与 DictWriter
确保使用 `newline=''`，否则 Windows 下可能在行间出现空白行。

In [19]:
import csv

data = [
    ['姓名', '年龄', '城市'],
    ['张三', 25, '北京'],
    ['李四', 30, '上海'],
    ['王五', 28, '广州'],
]

with open('out_basic.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data)
print('写入 out_basic.csv 完成')

# 使用 DictWriter
rows = [
    {'姓名': '张三', '年龄': 25, '城市': '北京'},
    {'姓名': '李四', '年龄': 30, '城市': '上海'},
]
with open('out_dict.csv', 'w', encoding='utf-8', newline='') as f:
    fieldnames = ['姓名', '年龄', '城市']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)
print('写入 out_dict.csv 完成')

写入 out_basic.csv 完成
写入 out_dict.csv 完成


### 3.1 引号与分隔符控制
通过 `dialect` 或参数：`delimiter`, `quotechar`, `quoting`, `escapechar`。
常见 quoting 模式：
- `csv.QUOTE_MINIMAL` (默认，仅必要时加引号)
- `csv.QUOTE_ALL` (所有字段加引号)
- `csv.QUOTE_NONNUMERIC` (非数字加引号，同时读入时数字自动转 float)
- `csv.QUOTE_NONE` (禁用引号，需要配合 `escapechar`)

In [20]:
import csv
data = [ ['a,b', 1], ['c\n换行', 2] ]
with open('out_quotes.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writerow(['col1', 'col2'])
    writer.writerows(data)
print('写入 out_quotes.csv 完成')

写入 out_quotes.csv 完成


## 4. 自动检测分隔符：Sniffer
当不确定文件分隔符时使用 `csv.Sniffer().sniff(sample)`。注意：需提供足够行样本。

In [21]:
import csv
with open('sample.csv', 'r', encoding='utf-8', newline='') as f:
    sample = f.read(2048)
    f.seek(0)
    dialect = csv.Sniffer().sniff(sample, delimiters=',;\t|')
    reader = csv.reader(f, dialect)
    for row in reader:
        print(row)

['姓名', '年龄', '城市']
['张三', '25', '北京']
['李四', '30', '上海']
['王五', '28', '广州']


## 5. 大文件与流式处理
内置 `csv` 是迭代器方式，天然流式；`pandas` 需要 `chunksize` 分块。避免一次读入超大文件占满内存。

In [22]:
import pandas as pd
chunk_iter = pd.read_csv('sample.csv', chunksize=10000)  # 每 1 万行一块
total = 0
for chunk in chunk_iter:
    total += len(chunk)
print('总行数估计:', total)

总行数估计: 3


## 6. 使用 pandas 读取与写入 CSV
`pandas.read_csv` 支持极多参数：`sep`, `encoding`, `dtype`, `usecols`, `nrows`, `skiprows`, `parse_dates`, `na_values`, `keep_default_na`。
写入使用 `DataFrame.to_csv(index=False, encoding='utf-8')`。

In [23]:
import pandas as pd
df = pd.read_csv('sample.csv', encoding='utf-8')
print(df.head())

# 指定列类型 & 只读部分列
df_subset = pd.read_csv('sample.csv', usecols=['姓名', '城市'], dtype={'城市': 'string'})
print(df_subset)

# 写入新文件
df.to_csv('out_pandas.csv', index=False, encoding='utf-8')
print('写入 out_pandas.csv 完成')

   姓名  年龄  城市
0  张三  25  北京
1  李四  30  上海
2  王五  28  广州
   姓名  城市
0  张三  北京
1  李四  上海
2  王五  广州
写入 out_pandas.csv 完成


### 6.1 处理缺失值与日期
可用 `na_values=['', 'NA']` 自定义缺失，`parse_dates=['创建时间']` 自动转为 datetime。

In [26]:
import pandas as pd
df_dates = pd.read_csv('sample.csv', parse_dates=['年龄'], na_values=['', 'NA'])
print(df_dates.dtypes)
print(df_dates.head())

姓名    object
年龄    object
城市    object
dtype: object
   姓名  年龄  城市
0  张三  25  北京
1  李四  30  上海
2  王五  28  广州


C:\Users\ocean\AppData\Local\Temp\ipykernel_8632\420998116.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_dates = pd.read_csv('sample.csv', parse_dates=['年龄'], na_values=['', 'NA'])


## 7. 压缩格式 (gzip) 读取与写入
CSV 常与 gzip 搭配：减少磁盘与网络占用。内置模块与 pandas 都支持。

In [27]:
import gzip, csv
# 写入压缩
with gzip.open('data.csv.gz', 'wt', encoding='utf-8', newline='') as f:
    w = csv.writer(f)
    w.writerow(['id', 'value'])
    for i in range(5):
        w.writerow([i, f'v{i}'])

# 读取压缩
with gzip.open('data.csv.gz', 'rt', encoding='utf-8', newline='') as f:
    r = csv.reader(f)
    for row in r:
        print(row)

# pandas 直接读
import pandas as pd
pdf = pd.read_csv('data.csv.gz', compression='gzip')
print(pdf)

['id', 'value']
['0', 'v0']
['1', 'v1']
['2', 'v2']
['3', 'v3']
['4', 'v4']
   id value
0   0    v0
1   1    v1
2   2    v2
3   3    v3
4   4    v4


## 8. 性能优化要点
- 读大文件优先：
  - 内置 csv 流式迭代 + 自己处理逻辑
  - pandas `chunksize` 分块聚合
- 指定 `dtype` 避免类型反复推断
- 仅选所需列：`usecols`
- 避免在循环内频繁 `DataFrame.append`，改用收集列表后一次 `pd.DataFrame(list)`
- 写入时禁用索引：`index=False`
- 多进程/多线程：IO 密集读写可用 concurrent.futures + 分片 (注意顺序)


## 9. 常见坑与错误排查
| 问题 | 原因 | 解决 |
|------|------|------|
| 读取出现额外空行 | 未使用 `newline=''` (Windows) | 打开文件时加 `newline=''` |
| 中文乱码 | 编码不匹配 (GBK vs UTF-8) | 用正确 `encoding`，必要时尝试 `utf-8-sig` |
| 分隔错位 | 字段含逗号但无引号 | 写入使用正确 quoting；读取时检查数据源 |
| Sniffer 失败 | 样本太小或数据不规则 | 增加 sample 长度或手动指定分隔符 |
| pandas 占用内存大 | 一次读入巨文件 | 使用 `chunksize` 分块 |
| Excel 打开格式错乱 | 分隔符/编码不兼容 | 导出为 UTF-8-BOM 或使用制表符分隔 |
| 数字被当成科学计数/公式 | Excel 自动格式化 | 写入前加前缀或另存为文本格式 |
| CSV 注入风险 | 单元格以 `= + - @` 开头 | 前缀 `'` 转义或过滤这些前缀 |

## 10. 安全：CSV 注入防护
当导出给用户在 Excel 打开时，恶意者可能在单元格写 `=HYPERLINK(...)` 或 `=cmd|'...'!A0`。防护策略：
- 若单元格以 `[= + - @]` 开头，前置 `'` (单引号) 或空格
- 白名单：仅允许可打印普通字符
- 输出前统一清洗函数


In [ ]:
def sanitize_csv_cell(value: str) -> str:
    prefix_chars = ('=', '+', '-', '@')
    if isinstance(value, str) and value.startswith(prefix_chars):
        return '\'' + value  # Excel 中显示原样
    return value

row = ['=CMD', '正常', '@attack']
cleaned = [sanitize_csv_cell(v) for v in row]
print(cleaned)

## 11. 自定义 Dialect 注册
可以注册一个常用格式，复用。

In [ ]:
import csv
csv.register_dialect('pipe', delimiter='|', quotechar='"', quoting=csv.QUOTE_MINIMAL)
with open('pipe.csv', 'w', encoding='utf-8', newline='') as f:
    w = csv.writer(f, dialect='pipe')
    w.writerow(['a', 'b'])
    w.writerow(['1', '2'])
print(open('pipe.csv', 'r', encoding='utf-8').read())

## 12. 与 JSON / Excel 对比简表
| 维度 | CSV | JSON | Excel (.xlsx) |
|------|-----|------|---------------|
| 结构 | 平面二维 | 嵌套灵活 | 多表/样式/公式 |
| 体积 | 小 (文本) | 中等 | 较大 (压缩包) |
| 解析 | 极快/简单 | 需解析器 | 需专用库 |
| 表达力 | 低 | 高 | 高 (含格式) |
| 流式处理 | 容易 | 容易 | 相对麻烦 |
| 适用场景 | 快速数据交换 | API / 配置 / 嵌套 | 报表 / 人工查看 |

## 13. 总结与建议
- 小数据 & 简单结构：内置 csv 模块
- 需要清洗/分析：pandas
- 大数据：流式迭代 + 分块 + 压缩
- 导出给非技术人员：注意编码与 Excel 兼容 (UTF-8-BOM)
- 安全：防公式注入；审计来源数据
- 规范：固定列顺序 + 明确类型 + 必要时提供 schema (文档)
